In [3]:
import numpy as np
import os

In [4]:
print(os.getcwd())
os.chdir('/usr/users/fatma.chafra01/ColabDesign')
print(os.getcwd())
#can't find colabdesign script if not in ColabDesign main directory

/home/mpg01/MBPC/fatma.chafra01/ColabDesign/af/examples
/home/mpg01/MBPC/fatma.chafra01/ColabDesign


In [5]:
from colabdesign.af.alphafold.common import residue_constants

In [6]:
# forming the bias matrix to fix the nb core positions
# total sequence length of nb is: 127
total_seq = "MAEVQLVESGGGLVQPGGSLRLSCTTSTSLFSITTMGWYRQAPGKQRELVASIKRGGGTNYADSMKGRFTISRDNARNTVFLEMNNLTTEDTAVYYCNAAILAYTGEVTNYWGQGTQVTVSSGQAGQ"
print('total seq length', len(total_seq))
# structurally represented seq length 117 (including the 7 residue gap in between the two sides of the structure) 
seq = 'VQLVESGGGLVQPGGSLRLSCTTSTSLFSITTMGWYRQAPGKQRELVASIKRGGGTNYADSMKGRFTISRDNARNTVFLEMNNLTTEDTAVYYCNAAILAYTGEVTNYWGQGTQVTV'
print('seq length in structure', len(seq))
# make sure to also delete the last part that is not in the structure from the original sequence because it will not be present in the later generated sequence
print('seq length', len(seq))


total seq length 127
seq length in structure 117
seq length 117


In [5]:
# START OF GENERATING THE BIAS MATRIX FOR THE ENTIRE CDRS (INITIAL BIAS MATRIX AFTER THE FIX!!)
# previously, thought that there would be a truncation of 19 aa from the start of the sequence upto the point of the first aa of the longest uninterrupted stretch (19-120 positions)
# because of the flanking missing structures of the sequence, the aa seq got truncated so had to change the intervals from (1, 27), (35,53), (59,99), (112,127) to:
# fix_pos = [(19, 27), (35,53), (59,99), (112,120)]
# but this is wrong because then the binder length should have been 120-19 +1 = 102 but it is 117!
# fixing this means removing the first 3 aa that are flanking from the first interval and the last 7 amino acids that are flanking from the last interval:
fix_pos = [(4, 27), (35,53), (59,99), (112,120)]
# then have to restart the indeces from 1 because the first aa is the fourth aa (subtract 3 from each position):
fix_pos = [(1, 24), (32,50), (56,96), (109,117)]
print(fix_pos)

[(1, 24), (32, 50), (56, 96), (109, 117)]


In [8]:
# instead trying to make a bias matrix as suggested here: https://github.com/sokrypton/ColabDesign/issues/107
binder_len = 117
# get a bias matrix of the dimensions (no of rows: binder length, no of cols: all 20 amino acids as a one hot)
bias = np.zeros((binder_len,20))
for item in fix_pos:
  start = item[0] -1
  end = item[1] -1
  print(start, end)
  while start <= end:
    aa = seq[start]
    print(start, aa)
    bias[start,residue_constants.restype_order[str(aa)]] = 1e8
    print(f'bias added to:{start} as {aa}')
    start += 1
    # because the index changed once the pdb file was truncated
    # bias[start-19,residue_constants.restype_order[str(aa)]] = 1e8
    # order comes from residue_constants.restype_order[str(aa)]
    
print('bias matrix', bias)

1 116
1 Q
bias added to:1 as Q
2 L
bias added to:2 as L
3 V
bias added to:3 as V
4 E
bias added to:4 as E
5 S
bias added to:5 as S
6 G
bias added to:6 as G
7 G
bias added to:7 as G
8 G
bias added to:8 as G
9 L
bias added to:9 as L
10 V
bias added to:10 as V
11 Q
bias added to:11 as Q
12 P
bias added to:12 as P
13 G
bias added to:13 as G
14 G
bias added to:14 as G
15 S
bias added to:15 as S
16 L
bias added to:16 as L
17 R
bias added to:17 as R
18 L
bias added to:18 as L
19 S
bias added to:19 as S
20 C
bias added to:20 as C
21 T
bias added to:21 as T
22 T
bias added to:22 as T
23 S
bias added to:23 as S
24 T
bias added to:24 as T
25 S
bias added to:25 as S
26 L
bias added to:26 as L
27 F
bias added to:27 as F
28 S
bias added to:28 as S
29 I
bias added to:29 as I
30 T
bias added to:30 as T
31 T
bias added to:31 as T
32 M
bias added to:32 as M
33 G
bias added to:33 as G
34 W
bias added to:34 as W
35 Y
bias added to:35 as Y
36 R
bias added to:36 as R
37 Q
bias added to:37 as Q
38 A
bias add

In [10]:
# check whether the bias matrix makes sense by randomly printing out a row that has to contain a serine (pos 19 according to prev numbering, now pos 19 - 3 so 16)
print(bias[16,:])
print(bias[0,:])
# I don't know the order of the one hot but it is not exactly the alphabetically ordered classical one hot
def find_rows_without_value(arr, value):
    # Check each row for the presence of the value
    rows_with_value = np.any(np.isclose(arr, value), axis=1)

    # Get the indices of rows that don't have the value
    rows_without_value = np.where(~rows_with_value)[0]

    return rows_without_value

print("Rows without 1.0e+08 in bias matrix:", find_rows_without_value(bias, 1.0e+08))

[0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 1.e+08 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Rows without 1.0e+08 in bias matrix: [0]


In [11]:
# check the bias matrix shape:
bias.shape == (binder_len,20)

True

In [14]:
# write a function to output the bias matrix into a csv file with filename bias_matrix_8ee2.csv
# Save the matrix to a CSV file
np.savetxt("bias_matrix_8ee2.csv",bias, delimiter=",", fmt="%.2f")

In [19]:
# SINGLE POINT MUTATION BIAS MATRIX (AFTER DISCUSSION WITH DIRK)
# example case where only the first aa is changed
fix_pos = [(2,117)]
to_be_mutated = residue_constants.restype_order['A']
# try to fix the amino acid to be present in this specific residue
print(residue_constants.restype_order['A'])
print(residue_constants.restype_order['G'])
print(residue_constants.restype_order['R'])
print(residue_constants.restype_order['K'])
print(find_rows_without_value(bias, 1.0e+08))
print(type(find_rows_without_value(bias, 1.0e+08)))
# add the bias for that specific residue (to_be_mutated)
print(bias[find_rows_without_value(bias, 1.0e+08),:])
bias[find_rows_without_value(bias, 1.0e+08),to_be_mutated] = 1.0e+08
print(find_rows_without_value(bias, 1.0e+08))
print(bias[0,:])


0
7
1
11
[0]
<class 'numpy.ndarray'>
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[]
[1.e+08 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]


In [ ]:
def generate_bias_matrix(seq, fix_pos, to_be_mutated=None):
    '''
    seq string of the amino acid sequence that is to be used (after the flanking non-structure associated regions are removed)
    fix_pos list of regions (tuples) to be kept constant in the seq (not zero indexed!)
    to_be_mutated list of amino acid strings (single letter) to mutate the non-fix position residues (in order of the non-fix position residues in seq)
    '''
    binder_len = len(seq)
    # get a bias matrix of the dimensions (no of rows: binder length, no of cols: all 20 amino acids as a one hot)
    bias = np.zeros((binder_len,20))
    for item in fix_pos:
        start = item[0] -1
        end = item[1] -1
        print(start, end)
        while start <= end:
            aa = seq[start]
            print(start, aa)
            bias[start,residue_constants.restype_order[str(aa)]] = 1e8
            print(f'bias added to:{start} as {aa}')
            start += 1
            # because the index changed once the pdb file was truncated
            # bias[start-19,residue_constants.restype_order[str(aa)]] = 1e8
            # order comes from residue_constants.restype_order[str(aa)]
    rows_to_change = find_rows_without_value(bias, 1.0e+08)
    print('rows_to_change:', rows_to_change)
    
    if to_be_mutated != None:
        index_to_be_mutated = [residue_constants.restype_order[i] for i in to_be_mutated]
        print('index to be mutated:', index_to_be_mutated)
        if len(rows_to_change) != len(index_to_be_mutated):
            print('Number of positions to mutate and number of specified amino acids do not match!')
            return 
        for i in range(len(rows_to_change)):
            bias[rows_to_change[i],index_to_be_mutated[i]] = 1.0e+08


    print('bias matrix', bias)
    return bias


In [39]:
test = generate_bias_matrix('VQLVESGGGLVQPGGSLRLSCTTSTSLFSITTMGWYRQAPGKQRELVASIKRGGGTNYADSMKGRFTISRDNARNTVFLEMNNLTTEDTAVYYCNAAILAYTGEVTNYWGQGTQVTV', [(2,117)], to_be_mutated=None)
test1 = generate_bias_matrix('VQLVESGGGLVQPGGSLRLSCTTSTSLFSITTMGWYRQAPGKQRELVASIKRGGGTNYADSMKGRFTISRDNARNTVFLEMNNLTTEDTAVYYCNAAILAYTGEVTNYWGQGTQVTV', [(2,117)], to_be_mutated=['A'])
test3 = generate_bias_matrix('VQLVESGGGLVQPGGSLRLSCTTSTSLFSITTMGWYRQAPGKQRELVASIKRGGGTNYADSMKGRFTISRDNARNTVFLEMNNLTTEDTAVYYCNAAILAYTGEVTNYWGQGTQVTV', [(2,100),(102,117)], to_be_mutated=['A','R'])

1 116
1 Q
bias added to:1 as Q
2 L
bias added to:2 as L
3 V
bias added to:3 as V
4 E
bias added to:4 as E
5 S
bias added to:5 as S
6 G
bias added to:6 as G
7 G
bias added to:7 as G
8 G
bias added to:8 as G
9 L
bias added to:9 as L
10 V
bias added to:10 as V
11 Q
bias added to:11 as Q
12 P
bias added to:12 as P
13 G
bias added to:13 as G
14 G
bias added to:14 as G
15 S
bias added to:15 as S
16 L
bias added to:16 as L
17 R
bias added to:17 as R
18 L
bias added to:18 as L
19 S
bias added to:19 as S
20 C
bias added to:20 as C
21 T
bias added to:21 as T
22 T
bias added to:22 as T
23 S
bias added to:23 as S
24 T
bias added to:24 as T
25 S
bias added to:25 as S
26 L
bias added to:26 as L
27 F
bias added to:27 as F
28 S
bias added to:28 as S
29 I
bias added to:29 as I
30 T
bias added to:30 as T
31 T
bias added to:31 as T
32 M
bias added to:32 as M
33 G
bias added to:33 as G
34 W
bias added to:34 as W
35 Y
bias added to:35 as Y
36 R
bias added to:36 as R
37 Q
bias added to:37 as Q
38 A
bias add

In [43]:
print(test[0,:])
print(test[116,:])
print(test1[0,:])
print(find_rows_without_value(test1,1e8))
print(test3[0,:])
print(test3[100,:])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 1.e+08]
[1.e+08 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
[]
[1.e+08 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
[0.e+00 1.e+08 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00
 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]


In [ ]:
# SECOND BIAS MATRIX FOR THE 3A CONTACT POSITIONS (canceled)
# positions in 3A distance: S/32, T/34, T/35, K/54, R/55, L/102, Y/104 for 8ee2.pdb
# so the fix intervals would be: (1,31), (33,34?), (36,53), (56,101), (193,104?), (105,120)
# removing the first 3 aa that are flanking from the first interval and the last 7 amino acids that are flanking from the last interval:
fix_pos = [(4, 27), (35,53), (59,99), (112,120)]
# then have to restart the indeces from 1 because the first aa is the fourth aa (subtract 3 from each position):
fix_pos = [(1, 24), (32,50), (56,96), (109,117)]
print(fix_pos)